### Init Setup

In [2]:
import trimesh
import os
import torch
import csv
import numpy as np
import pandas as pd
from datetime import datetime
from chamferdist import ChamferDistance
from torch.utils.data import DataLoader
from shap_e.models.testing_utils import test_model
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.fine_tune_data import Abstraction_Dataset
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, decode_latent_mesh

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

MODEL_PATH="/storage/etaisella/repos/shape_proj/outputs/abstrction_planes_22/model_epoch_24.pt"
DATASET_PATH="/storage/etaisella/repos/shape_proj/data/abstract_dataset/test_original"
PLOT_FOLDER="/storage/etaisella/repos/shape_proj/outputs/abstrction_planes_22/meshes"

NUM_CONTROL_LAYERS=24
GUIDANCE_SCALE=7.5
NUM_SAMPLES=4096

def prompt2filename(prompt: str):
    filename = prompt.replace(" ", "_")
    filename = filename.replace("?", "")
    filename = filename.replace("!", "")
    filename = filename.replace(",", "")
    filename = filename.replace('\"', '')
    filename = filename.replace('\\', '')
    filename = filename.replace('/', '')
    return filename

#mesh = trimesh.load_mesh('/storage/etaisella/repos/CuboidAbstractionViaSeg/PretrainModels/airplane/infer/test/1a9b552befd6306cc8f2d5fe7449af61_cube_masked.ply')
#print(type(mesh))
#mesh.show()

cuda


### Load Models

In [2]:
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
model.wrapped.backbone.make_ctrl_layers(NUM_CONTROL_LAYERS, 
                                            reverse=False,
                                            cross_mode=True,
                                            conditional=True,
                                            no_one_conv=True)
model.wrapped.set_up_controlnet_cond()
model.load_state_dict(torch.load(MODEL_PATH))
diffusion = diffusion_from_config(load_config('diffusion'))

### Setup Dataset

In [3]:
val_dataset = Abstraction_Dataset(DATASET_PATH, verbose=True)
val_dataloader = DataLoader(
                val_dataset, 
                batch_size = 1,
                shuffle=False, 
                num_workers=0
            )
print("Dataset size: ", len(val_dataset))

dataset length:  808
Dataset size:  808


### Get Meshes

In [10]:
# create output dir if it doesn't exist
os.makedirs(PLOT_FOLDER, exist_ok=True)
cond_latent = 'latent_abstraction'
target_latent = 'latent_original'
with torch.no_grad():
    for sample in val_dataloader:
        if sample == -1:
            print(f"issue loading validation sample")
            break
        prompt = sample['prompt'][0]
        filename = prompt2filename(prompt)
        filepath = os.path.join(PLOT_FOLDER, filename)

        # Extracting Output Mesh
        print(f"creating output mesh for prompt: {prompt}")

        out_latent = test_model(model=model,
                                diffusion=diffusion, 
                                xm=xm,
                                output_folder=PLOT_FOLDER,
                                cond=sample[cond_latent][0].to(device).detach(),
                                epoch=0, 
                                prompt=prompt,
                                device=device,
                                guidance_scale=GUIDANCE_SCALE,
                                return_latents=True)
        t = decode_latent_mesh(xm, out_latent).tri_mesh()
        with open(f'{filepath}.ply', 'wb') as f:
            t.write_ply(f)
        with open(f'{filepath}.obj', 'w') as f:
            t.write_obj(f)
        
        # Extracting Condition Mesh
        print(f"creating condition mesh for prompt: {prompt}")
        t = decode_latent_mesh(xm, sample[cond_latent][0].to(device).detach()).tri_mesh()
        with open(f'{filepath}_cond.ply', 'wb') as f:
            t.write_ply(f)
        with open(f'{filepath}_cond.obj', 'w') as f:
            t.write_obj(f)


    # clear cache
    del sample
    torch.cuda.empty_cache()

creating output mesh for prompt: jet fighter


  0%|          | 0/64 [00:00<?, ?it/s]

### Calc Metrics on Meshes

In [ ]:
chamferDist = ChamferDistance()

# load all <mesh>.ply and <mesh>_cond.ply in plot folder
# sample NUM_SAMPLES points from each mesh
# calculate chamfer distance between points

for filename in os.listdir(PLOT_FOLDER):
    if filename.endswith("cond.ply"):
        cond_mesh = trimesh.load_mesh(os.path.join(PLOT_FOLDER, filename))
        # load non cond mesh
        try:
            out_mesh = trimesh.load_mesh(os.path.join(PLOT_FOLDER, filename[:-9] + ".ply"))
        except:
            print(f"no out mesh for {filename[:-9]}")
            continue
        # sample points
        sample_cond = torch.Tensor(trimesh.sample.sample_surface(cond_mesh, NUM_SAMPLES)[0]).to(device)
        sample_out = torch.Tensor(trimesh.sample.sample_surface(out_mesh, NUM_SAMPLES)[0]).to(device)

        # calculate chamfer distance
        dist_bidirectional = chamferDist(sample_cond.unsqueeze(0), sample_out.unsqueeze(0), bidirectional=True)

        print(f"ChamferDistance for {filename[:-9]}: {dist_bidirectional}")

### Calc Metrics Originals

In [4]:
chamferDist = ChamferDistance()

abstractions_folder = '/storage/etaisella/repos/CuboidAbstractionViaSeg/PretrainModels/airplane/infer/test'
#originals_folder = '/storage/etaisella/repos/shapenetcore-glb/airplane'
originals_folder = '/storage/etaisella/ShapeNetNormal4096/airplane'
csv_path = '/storage/etaisella/repos/shape_proj/data/abstraction/airplane_meta.csv'

# get abstraction data paths
abstraction_data_paths = os.listdir(abstractions_folder)
# filter out elements that do not end with 'cube_masked.ply'
abstraction_data_paths = [path for path in abstraction_data_paths if path.endswith('cube_masked.ply')]
# check if abstraction ids are found in original data
ids_found_in_both = []
objects_for_ids = {}
original_data_paths = os.listdir(originals_folder)
for path in abstraction_data_paths:
    abstraction_id = path.split('_')[0]
    #if f'{abstraction_id}.glb' in original_data_paths:
    if f'{abstraction_id}.npy' in original_data_paths:
        object_dict = {}
        #object_dict['original_path'] = os.path.join(originals_folder, f'{abstraction_id}.glb')
        object_dict['original_path'] = os.path.join(originals_folder, f'{abstraction_id}.npy')
        object_dict['abstraction_path'] = os.path.join(abstractions_folder, path)
        ids_found_in_both.append(abstraction_id)
        objects_for_ids[abstraction_id] = object_dict
print(f'Number of abstraction ids found in both abstractions and originals: {len(ids_found_in_both)}')
# check found ids metadata
found_in_metadata = []
with open(csv_path, newline='') as csvfile:
    spamreader = csv.reader(csvfile)
    for i, row in enumerate(spamreader):
        if i == 0:
            print(row)
            id_offset = row.index('fullId')
            name_offset = row.index('name')
            tags_offset = row.index('tags')
        else:
            obj_id = row[id_offset].split('.')[-1]
            if obj_id in ids_found_in_both:
                object_dict = {}
                object_dict['original_path'] = objects_for_ids[obj_id]['original_path']
                object_dict['abstraction_path'] = objects_for_ids[obj_id]['abstraction_path']
                object_dict['id'] = obj_id
                object_dict['name'] = row[name_offset]
                object_dict['tags'] = row[tags_offset]
                found_in_metadata.append(object_dict)
                print(row[id_offset], row[name_offset], row[tags_offset])
print(f'Number of abstraction ids found in metadata: {len(found_in_metadata)}')

total_dist = 0
for obj in found_in_metadata:
    cond_mesh = trimesh.load_mesh(obj['abstraction_path'])
    # load non cond mesh
    print(obj['original_path'])
    #try:
    #    out_mesh = trimesh.load_mesh(object['original_path'])
    #except:
    #    print(f"no out mesh for {object['name']}")
    #    continue
    #out_mesh = trimesh.util.concatenate(
    #            tuple(trimesh.Trimesh(vertices=g.vertices, faces=g.faces)
    #                for g in out_mesh.geometry.values()))
    # sample points
    sample_cond = torch.Tensor(trimesh.sample.sample_surface(cond_mesh, NUM_SAMPLES)[0]).to(device)
    sample_out = torch.from_numpy(np.load(obj['original_path'])).float()[:, :3].to(device)
    print(sample_out.shape)
    #sample_out = torch.Tensor(trimesh.sample.sample_surface(out_mesh, NUM_SAMPLES)[0]).to(device)
    # calculate chamfer distance
    dist_bidirectional = chamferDist(sample_cond.unsqueeze(0), sample_out.unsqueeze(0), bidirectional=True)
    #dist_bidirectional = chamferDist(sample_cond.unsqueeze(0), sample_out.unsqueeze(0))
    total_dist += dist_bidirectional
    print(f"ChamferDistance for {obj['name']}: {dist_bidirectional}")

total_dist /= len(found_in_metadata)
print(f"Average ChamferDistance: {total_dist}")

Number of abstraction ids found in both abstractions and originals: 808
['fullId', 'wnsynset', 'wnlemmas', 'up', 'front', 'name', 'tags']
3dw.f26ea1a00455f44fb88e2a19106395c2 Jet 
3dw.b0b164107a27986961f6e1cef6b8e434 prototipo 002 
3dw.ce682d7a2bbf77b6fc4b92d3d335214a F-15 
3dw.ecbb6df185a7b260760d31bf9510e4b7 Hypersonic Jet 
3dw.3d23703a618ce7df1e569ed4e4cfe84 F-117 Stealth Fighter 
3dw.e25e3dc95243a92c59bcb260e54d3785 F/A-18C Blue Angels Paint 
3dw.bc7ead8b45952ab8822054a0a020bf4a Atreides ´Thopter on a Sand Storm 
3dw.fb402a36e91ab1b44e7761521d3c6953 Hover Jet Ride: Santa Ride 
3dw.97eb9cf6c8a11a389967b23b351d6841 VF-25F Messiah  "FIGHTER" 
3dw.3265b621ca222d29d00d52e62bf14ee9 MiG-31 Firefox 
3dw.a702da03d770f5096e2738fc9da60e6f Aerospace - Northrop F-5 Freedom Fighter 
3dw.b22014408721ec314567cadca15fe337 Military VTOL Aircraft (camo) 
3dw.9b687f9cff46d43d89c2da356f872ebc Private Jet 
3dw.4374a3b4b98e247b398db3ebdf468ed7 Jet Privée 
3dw.cb1aff81a3c6ef71d25fd5e3115979a5 stealth UAV 

In [24]:
def calc_abstraction_metrics_ref(abstraction_folder: str,
                                 originals_folder: str,
                                 csv_report_path: str,
                                 device: torch.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
                                 csv_metadata_path: str = '/storage/etaisella/repos/shape_proj/data/abstraction/airplane_meta.csv',
                                 biderictional: bool = True,
                                 originals_from_shapenet: bool = False,):
    chamferDist = ChamferDistance()
    # get abstraction data paths
    abstraction_data_paths = os.listdir(abstraction_folder)
    # filter out elements that do not end with 'cube_masked.ply'
    abstraction_data_paths = [path for path in abstraction_data_paths if path.endswith('cube_masked.ply')]
    print(f'Number of abstraction data paths: {len(abstraction_data_paths)}')

    # check if abstraction ids are found in original data
    ids_found_in_both = []
    original_data_paths = os.listdir(originals_folder)
    print(f'Number of original data paths: {len(original_data_paths)}')
    objects_for_ids = {}
    for path in abstraction_data_paths:
        abstraction_id = path.split('_')[0]
        if originals_from_shapenet:
            filename = f'{abstraction_id}.glb'
        else:
            filename = f'{abstraction_id}.npy'
        if filename in original_data_paths:
            object_dict = {}
            object_dict['original_path'] = os.path.join(originals_folder, filename)
            object_dict['abstraction_path'] = os.path.join(abstraction_folder, path)
            ids_found_in_both.append(abstraction_id)
            objects_for_ids[abstraction_id] = object_dict
    print(f'Number of abstraction ids found in both abstractions and originals: {len(ids_found_in_both)}')

    # check found ids metadata
    found_in_metadata = []
    with open(csv_metadata_path, newline='') as csvfile:
        spamreader = csv.reader(csvfile)
        for i, row in enumerate(spamreader):
            if i == 0:
                print(row)
                id_offset = row.index('fullId')
                name_offset = row.index('name')
                tags_offset = row.index('tags')
            else:
                obj_id = row[id_offset].split('.')[-1]
                if obj_id in ids_found_in_both:
                    object_dict = {}
                    object_dict['original_path'] = objects_for_ids[obj_id]['original_path']
                    object_dict['abstraction_path'] = objects_for_ids[obj_id]['abstraction_path']
                    object_dict['id'] = obj_id
                    object_dict['name'] = row[name_offset]
                    object_dict['tags'] = row[tags_offset]
                    found_in_metadata.append(object_dict)
                    print(row[id_offset], row[name_offset], row[tags_offset])
    print(f'Number of abstraction ids found in metadata: {len(found_in_metadata)}')

    # calculate metrics
    dataframe = {'ids':[], 'names':[], 'tags':[], 'chamfer_dist':[]}
    total_dist = 0
    for obj in found_in_metadata:
        cond_mesh = trimesh.load_mesh(obj['abstraction_path'])
        # load non cond mesh
        print(obj['original_path'])
        if originals_from_shapenet:
            try:
                out_mesh = trimesh.load_mesh(obj['original_path'])
            except:
                print(f"no out mesh for {obj['name']}")
                continue
            out_mesh = trimesh.util.concatenate(
                        tuple(trimesh.Trimesh(vertices=g.vertices, faces=g.faces)
                            for g in out_mesh.geometry.values()))
            sample_out = torch.Tensor(trimesh.sample.sample_surface(out_mesh, NUM_SAMPLES)[0]).to(device)
        else:
            sample_out = torch.from_numpy(np.load(obj['original_path'])).float()[:, :3].to(device)
        sample_cond = torch.Tensor(trimesh.sample.sample_surface(cond_mesh, NUM_SAMPLES)[0]).to(device)
        
        # calculate chamfer distance
        dist = chamferDist(sample_cond.unsqueeze(0), sample_out.unsqueeze(0), bidirectional=biderictional)
        total_dist += dist
        print(f"ChamferDistance for {obj['name']}: {dist}")

        # logging to csv
        dataframe['ids'].append(obj['id'])
        dataframe['names'].append(obj['name'])
        dataframe['tags'].append(obj['tags'])
        dataframe['chamfer_dist'].append(dist.cpu().numpy())


    total_dist /= len(found_in_metadata)
    total_dist = total_dist.cpu().numpy()
    print(f"Average ChamferDistance: {total_dist}")

    # log all metrics
    total_frame = {'time': [str(datetime.now())], 'total_calculated':[len(found_in_metadata)],'total_dist': [total_dist]}
    with open(csv_report_path,'a') as f:
        pd.DataFrame(dataframe).to_csv(f)
        f.write("\n")
        pd.DataFrame(total_frame).to_csv(f)

In [26]:
calc_abstraction_metrics_ref(abstraction_folder='/storage/etaisella/repos/CuboidAbstractionViaSeg/PretrainModels/airplane/infer/test',
                             originals_folder='/storage/etaisella/ShapeNetNormal4096/airplane',
                             csv_report_path='/storage/etaisella/repos/shape_proj/outputs/abstraction_metrics/abstraction_metrics_airplane.csv',)

Number of abstraction data paths: 808
Number of original data paths: 4045
Number of abstraction ids found in both abstractions and originals: 808
['fullId', 'wnsynset', 'wnlemmas', 'up', 'front', 'name', 'tags']
3dw.f26ea1a00455f44fb88e2a19106395c2 Jet 
3dw.b0b164107a27986961f6e1cef6b8e434 prototipo 002 
3dw.ce682d7a2bbf77b6fc4b92d3d335214a F-15 
3dw.ecbb6df185a7b260760d31bf9510e4b7 Hypersonic Jet 
3dw.3d23703a618ce7df1e569ed4e4cfe84 F-117 Stealth Fighter 
3dw.e25e3dc95243a92c59bcb260e54d3785 F/A-18C Blue Angels Paint 
3dw.bc7ead8b45952ab8822054a0a020bf4a Atreides ´Thopter on a Sand Storm 
3dw.fb402a36e91ab1b44e7761521d3c6953 Hover Jet Ride: Santa Ride 
3dw.97eb9cf6c8a11a389967b23b351d6841 VF-25F Messiah  "FIGHTER" 
3dw.3265b621ca222d29d00d52e62bf14ee9 MiG-31 Firefox 
3dw.a702da03d770f5096e2738fc9da60e6f Aerospace - Northrop F-5 Freedom Fighter 
3dw.b22014408721ec314567cadca15fe337 Military VTOL Aircraft (camo) 
3dw.9b687f9cff46d43d89c2da356f872ebc Private Jet 
3dw.4374a3b4b98e247b398d